In [29]:
import cv2
import numpy as np

def add_text_watermark(image_path, watermark_text, output_path, transparency=0.5):
    image = cv2.imread(image_path)

    # Setarea fontului si a dimensiunii textului watermark
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 10
    font_thickness = 20
    text_size = cv2.getTextSize(watermark_text, font, font_scale, font_thickness)[0]

    # Calcularea pozitiei textului pentru a fi centrat in imagine
    text_position = ((image.shape[1] - text_size[0]) // 2, (image.shape[0] - text_size[1]) // 2)

    # Adaugarea textului cu transparenta
    overlay = image.copy()
    cv2.putText(overlay, watermark_text, text_position, font, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)
    cv2.addWeighted(overlay, transparency, image, 1 - transparency, 0, image)

    cv2.imwrite(output_path, image)

image_path = 'forest.png'
output_path = 'forestWatermarked.png'
watermark_text = 'Watermark Example'

add_text_watermark(image_path, watermark_text, output_path, transparency=0.5)

In [46]:
def add_watermark(image_path, watermark_path, output_path, position='top-left'):
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    # Citirea watermark-ului cu transparenta
    watermark = cv2.imread(watermark_path, cv2.IMREAD_UNCHANGED)
    (h_wm, w_wm) = watermark.shape[:2]

    # Calcularea pozitiei in functie de locatia specificata
    if position == 'top-left':
        x_pos = 0
        y_pos = 0
    elif position == 'top-right':
        x_pos = w - w_wm
        y_pos = 0
    elif position == 'bottom-left':
        x_pos = 0
        y_pos = h - h_wm
    elif position == 'bottom-right':
        x_pos = w - w_wm
        y_pos = h - h_wm
    elif position == 'middle':
        x_pos = (w - w_wm) // 2
        y_pos = (h - h_wm) // 2
    else:
        raise ValueError("Invalid position. Supported positions: 'top-left', 'top-right', 'bottom-left', 'bottom-right', 'middle'.")

    # Verific daca watermark-ul incape pe imaginea principala
    if x_pos < 0 or y_pos < 0 or x_pos + w_wm > w or y_pos + h_wm > h:
        raise ValueError("Watermark exceeds image boundaries.")

    # Extrag canalul de transparenta al watermark-ului
    alpha_channel = watermark[:, :, 3] / 255.0

    # Cropez watermark-ul la dimensiunea imaginii dorite
    watermark = watermark[:h, :w, :3]

    # Calculez regiunea in care se va aplica watermark-ul
    roi = image[y_pos:y_pos + h_wm, x_pos:x_pos + w_wm]
    
    # Combinarea imaginii principale cu watermark-ul utilizand canalul de transparenta
    alpha_channel_expanded = np.expand_dims(alpha_channel, axis=-1)
    blended = (1.0 - alpha_channel_expanded) * roi + alpha_channel_expanded * watermark

    # Actualizarea regiunii din imaginea principala cu watermark-ul
    image[y_pos:y_pos + h_wm, x_pos:x_pos + w_wm] = blended.astype(np.uint8)

    cv2.imwrite(output_path, image)

image_path = 'forest.png'
watermark_path = 'logo.png'
output_path = 'forestLogoBottomRight.png'

add_watermark(image_path, watermark_path, output_path, position='bottom-right')

In [47]:
def get_average_color(image, position, window_size=5):
    (h, w) = image.shape[:2]

    # Extrag vecinatatea pixelului la pozitia data
    start_y = max(0, position[1] - window_size // 2)
    end_y = min(h, position[1] + window_size // 2 + 1)
    start_x = max(0, position[0] - window_size // 2)
    end_x = min(w, position[0] + window_size // 2 + 1)

    neighborhood = image[start_y:end_y, start_x:end_x]

    # Calculez culoarea medie
    average_color = np.mean(neighborhood, axis=(0, 1))

    return average_color.astype(np.uint8)

def add_watermark_dynamic_color(image_path, watermark_path, output_path, position='top-left', window_size=5):
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    watermark = cv2.imread(watermark_path, cv2.IMREAD_UNCHANGED)
    (h_wm, w_wm) = watermark.shape[:2]

    # Calcularea pozitiei in functie de locatia specificata
    if position == 'top-left':
        x_pos = 0
        y_pos = 0
    elif position == 'top-right':
        x_pos = w - w_wm
        y_pos = 0
    elif position == 'bottom-left':
        x_pos = 0
        y_pos = h - h_wm
    elif position == 'bottom-right':
        x_pos = w - w_wm
        y_pos = h - h_wm
    elif position == 'middle':
        x_pos = (w - w_wm) // 2
        y_pos = (h - h_wm) // 2
    else:
        raise ValueError("Invalid position. Supported positions: 'top-left', 'top-right', 'bottom-left', 'bottom-right', 'middle'.")

    # Verificare daca watermark-ul incape pe imaginea principala
    if x_pos < 0 or y_pos < 0 or x_pos + w_wm > w or y_pos + h_wm > h:
        raise ValueError("Watermark exceeds image boundaries.")

    # Calcularea noii culori pentru watermark
    new_color = get_average_color(image, (x_pos + w_wm // 2, y_pos + h_wm // 2), window_size)

    # Aplicarea noii culori la canalul RGB al watermark-ului
    watermark[:, :, :3] = new_color

    # Calcularea canalului de transparenta pentru watermark
    alpha_channel = watermark[:, :, 3] / 255.0

    # Combinarea imaginii principale cu watermark-ul folosind canalul de transparenta
    blended = (
        image[y_pos:y_pos + h_wm, x_pos:x_pos + w_wm].astype(float) * (1 - alpha_channel[:, :, None]) +
        watermark[:, :, :3].astype(float) * (alpha_channel[:, :, None])
    ).astype(np.uint8)

    # Actualizarea regiunii din imaginea principala cu watermark-ul
    image[y_pos:y_pos + h_wm, x_pos:x_pos + w_wm] = blended

    cv2.imwrite(output_path, image)

image_path = 'forest.png'
watermark_path = 'logo.png'
output_path = 'forestWatermarkedBottomRightColor.png'
position = 'bottom-right'

add_watermark_dynamic_color(image_path, watermark_path, output_path, position)